In [1]:
import sys,os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.cm as cmx

import openpiv.tools
import openpiv.process
import openpiv.scaling

from skimage import img_as_int
from os import walk

#this works apparently only for savefig stuff
mpl.rcParams['figure.figsize']=(16.0,10.0)    #(6.0,4.0)
mpl.rcParams['font.size']=12            #10 
mpl.rcParams['savefig.dpi']=400             #72 
mpl.rcParams['figure.subplot.bottom']=.1    #.125

mpl.rcParams['legend.numpoints'] = 1

plt.rc('font', family='serif')
plt.rc('text', usetex=True)

#inline Shit
#%matplotlib inline
%config InlineBackend.figure_format='svg'
%config InlineBackend.rc = {'figure.facecolor': 'white', 'figure.subplot.bottom': 0.125, 'figure.edgecolor': 'white', 'savefig.dpi': 300, 'figure.figsize': (12.0, 8.0), 'font.size': 10}

#GUi shit
#matplotlib tk

mpl.get_configdir()

%load_ext autoreload
%autoreload 2


/opt/python3.4env/lib/python3.4/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [30]:
# Alle file paths der bilder in eine liste lesen : files = ['../data/exp1/f1.jpg', ..., '../data/exp1/f10.jpg' ]
import glob
import re

fileNumberRegex = re.compile(".*?_(\d*)\.")
pathImages = '../../../data/experiment1/'

files = glob.glob(pathImages+'*.jpg')
fileNumbers = [ int(fileNumberRegex.match(ff).group(1)) for ff in files ];
images = dict(zip(fileNumbers,files))

imagesRange = (min(images.keys()),max(images.keys()))

processingRange = (781,782);

if( min(processingRange) < min(imagesRange) ): processingRange[0] = imagesRange[0];
if( max(processingRange) > max(imagesRange) ): processingRange[1] = imagesRange[1];

imagesRange = range(processingRange[0], processingRange[1]);


In [33]:


def saveResults(outputPath, filename, frame_a, frame_b, x,y,u,v,u2,v2,u3,v3):
    plt.close("all")
    # Plotten Figure
    fig = plt.figure(figsize=(15,8));

    ax0 = fig.add_subplot(131)
    ax0.set_autoscale_on(False)
    ax0.set_title("PIV Extended Search Area" )


    ax0.matshow( frame_a, cmap=cm.Greys_r );
    ax0.quiver( x, y, u, v , color='r')

    ax1 = fig.add_subplot(132, sharex=ax0, sharey=ax0)
    ax1.set_autoscale_on(False)
    ax1.set_title("Outlier Rejected" )

    ax1.matshow( frame_a, cmap=cm.Greys_r );
    ax1.quiver( x, y, u2, v2 , color='r')

    ax3 = fig.add_subplot(133, sharex=ax0, sharey=ax0)
    ax3.set_autoscale_on(False)
    ax3.set_title("Replaced Outlier" )

    ax3.matshow( frame_a, cmap=cm.Greys_r );
    ax3.quiver( x, y, u3, v3 , color='r')


    ax0.set_xlim([0,frame_a.shape[1]])
    ax0.set_ylim([0,frame_a.shape[0]])
    ax1.set_xlim([0,frame_a.shape[1]])
    ax1.set_ylim([0,frame_a.shape[0]])
    fig.tight_layout()

    fig.savefig(outputPath + filename)
    plt.clf()

def extractVelocityProfile(imagePathA, imagePathB, outputPath, filePrefix , pivSettings, scaleFactor):


        frame_a = img_as_int(openpiv.tools.imread(imagePathA)).astype(np.int32) # make picture from float64 to int16 (try to use int32 FIX!
        frame_b = img_as_int(openpiv.tools.imread(imagePathB)).astype(np.int32)

        # Plot bilder in Graustufen
#         plt.matshow( frame_a, cmap=cm.Greys_r );
#         plt.matshow( frame_b, cmap=cm.Greys_r );

        u, v, sig2noise = openpiv.process.extended_search_area_piv(frame_a, 
                                                                   frame_b, 
                                                                   **pivSettings['extentedSearchAreaPivSettings'])
        print(u,v)
        x, y = openpiv.process.get_coordinates( image_size=frame_a.shape, 
                                                window_size=pivSettings['extentedSearchAreaPivSettings']['window_size'], 
                                                overlap=pivSettings['extentedSearchAreaPivSettings']['overlap'] )

        u2 = u.copy();
        v2 = v.copy();
        u2, v2, mask = openpiv.validation.sig2noise_val( u2, v2, sig2noise, **pivSettings['sig2NoiseSettings'] )


        u3 = u2.copy();
        v3 = v2.copy();
        u3, v3 = openpiv.filters.replace_outliers( u3, v3, **pivSettings['replaceOutlierSettings'] )
        
        
        
        x_meter, y_meter, u3_meter, v3_meter = openpiv.scaling.uniform(x, y, u3, v3, scaling_factor = scaleFactor )
        
        #saveResults(outputPath, filePrefix , frame_a, frame_b, x,y,u,v,u2,v2,u3,v3)

In [34]:

spacing = 8;
pivSettings = {
                    "extentedSearchAreaPivSettings" : 
                    {    "window_size" : 64, 
                        "dt" : 1e-3,
                        "search_area_size" : 64,
                        "overlap" : 64 - spacing,
                        "sig2noise_method" : "peak2peak"}
                    ,
                    "sig2NoiseSettings" :
                        {"threshold" : 0.5}
                    ,
                    "replaceOutlierSettings" : 
                    {
                        "method" :'localmean',
                        "max_iter" : 10, 
                        "kernel_size"  : 2
                    }
                  }

for i in imagesRange:
    extractVelocityProfile(images[i], images[i+1], "./", "exp1_vel" + str(i), pivSettings);


[[-32.80258179 -33.18405151 -27.51159477 ...,   8.94165039  12.82501125
    0.        ]
 [-27.21023369 -29.5600872  -24.27673149 ...,  11.13128567  11.30676174
    0.        ]
 [-37.09411621 -38.13934326 -30.88378716 ...,   5.76782179   9.38415527
    0.        ]
 ..., 
 [ -3.44467139  -3.86428809  -7.29751539 ...,  -0.53787231  -1.88446033
    0.        ]
 [-11.32202053 -12.27569485  -8.7928772  ...,   3.62396216   5.32531691
    0.        ]
 [  0.           0.           0.         ...,   0.           0.           0.        ]] [[ 16.26586914  16.00265503  19.10018921 ...,   5.97000074   6.98089552
    0.        ]
 [ 19.53887939  20.88928032  23.87237358 ...,  11.28768826  14.12963772
    0.        ]
 [ 23.12469292  24.00588799  27.56500053 ...,   3.46755958   3.63159156
    0.        ]
 ..., 
 [-12.74108791 -16.19720459 -14.7094717  ...,  10.07842922   4.11224365
    0.        ]
 [-12.29858303 -14.73998928 -15.47241116 ...,  14.91165066   7.20977736
    0.        ]
 [  0.           0.